# Data Science Toolbox: Literature Review
By Shanglin Zou and Daniel Jones

We chose to divide the work equally, resulting in a corresponding split of equity:
  - Shanglin Zou: 50%
  - Daniel Jones: 50%

---


As part of a literature review, our key aim was to survey both the current and historical landscape of data science and it's relation to cyber-security. This covers a broad set of topics, which we broke down into the following three categories [1]:
  1. Classical Statistics
  2. Machine Learning
  3. Neural Networks
  
We will aim to implement approaches from a couple of these fields in the practical part of this report.
 
TODO: talk about literature

### What are the broad types of data?
### What are the main types of resource?
We looked at:  books, papers, online tutorials, public code and data repositories.

Books:
 - "Introduction to Machine Learning"
 
Papers:
 
 - "Visual Analytics for Model Selection in Time Series Analysis"
 
Take-aways:
  - There are lots of resources on statistics, machine learning and neural networks (within books, papers and online resources) but not many that apply this to cyber-security.
  - secrepo.com proved a valuable resource for both data sets, and analysis of those data sources.
  - academic papers provided a combination of the two (stats applied to cyber-security data) but code samples were hard to come by
  
  
### What type of problems can the resources solve?

### Are there any generic data science resources that might be applicable? In what sense are they applicable?
Since there is little introductory material on data science applied to cyber-security, the generic data science resources proved useful to learn the theory behind it's application. Approach: read papers with direct application to cyber-security, and use generic data science resources to fill in knowledge gaps.

"Introduction to Machine learning" proved to be a useful reference text to learn about k-Nearest-Neighbors classification, since the papers applying it to cyber-security assumed knowledge of the algorithm as a pre-requisite.

"Visual Analytics for Model Selection in Time Series Analysis" is a good paper to learn model selection in Time series Analysis. The paper shows the steps we need to do when we try to model a time series data. 


### How might the approach be compared to other approaches, and/or applied across different datasets?
For time series approach, we can easily applied it across different datasets. Resampling, testing stationary, differencing, model selection, fitting and prediction are the steps we need to do. However, i think it does not work for analyzing string data(i.e Ip address, Id).

## Time-Series Analysis
(ARMA)

## Classification
(k-NN)

# References
[1]: [Data Science Toolbox, Lecture 01](https://www.ole.bris.ac.uk/bbcswebdav/pid-3554286-dt-content-rid-9984867_2/xid-9984867_2). Daniel Lawson.